In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_data_dir = 'dataset'
batch_size = 32
img_size = (128, 128)

In [3]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 1062 images belonging to 3 classes.


In [4]:

validation_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 264 images belonging to 3 classes.


In [5]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax')) 

2023-12-30 01:08:40.343407: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/10
33/33 [==============================] - 29s 855ms/step - loss: 0.8589 - accuracy: 0.5893 - val_loss: 0.6414 - val_accuracy: 0.7109
Epoch 2/10
33/33 [==============================] - 29s 865ms/step - loss: 0.5686 - accuracy: 0.7534 - val_loss: 0.5044 - val_accuracy: 0.8086
Epoch 3/10
33/33 [==============================] - 41s 1s/step - loss: 0.4945 - accuracy: 0.8233 - val_loss: 0.6067 - val_accuracy: 0.7578
Epoch 4/10
33/33 [==============================] - 33s 986ms/step - loss: 0.3952 - accuracy: 0.8573 - val_loss: 0.3674 - val_accuracy: 0.8789
Epoch 5/10
33/33 [==============================] - 31s 919ms/step - loss: 0.3257 - accuracy: 0.8893 - val_loss: 0.4831 - val_accuracy: 0.7930
Epoch 6/10
33/33 [==============================] - 29s 874ms/step - loss: 0.2644 - accuracy: 0.9204 - val_loss: 0.3615 - val_accuracy: 0.8789
Epoch 7/10
33/33 [==============================] - 29s 870ms/step - loss: 0.2435 - accuracy: 0.9223 - val_loss: 0.3353 - val_accuracy: 0.8828
Ep

In [8]:
model.save('mymodel.h5')

In [27]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the trained model
model = tf.keras.models.load_model('mymodel.h5')

# Function to preprocess the input image
def preprocess_image(image_path, img_size=(128, 128)):
    img = image.load_img(image_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the image
    return img_array

# Function to make predictions
def predict_image_class(image_path):
    img_array = preprocess_image(image_path)
    predictions = model.predict(img_array)
    class_index = np.argmax(predictions)
    
    # Assuming class indices 0, 1, 2 correspond to Buildings, Forest, Sea
    class_labels = {0: 'Buildings', 1: 'Forest', 2: 'Sea'}
    predicted_class = class_labels[class_index]
    
    return predicted_class, predictions[0]

# Test the model on a new image
image_path_to_test = 'dataset/forest/1004.jpg'  # Replace with the path to your test image
predicted_class, confidence = predict_image_class(image_path_to_test)

print(f"Predicted class: {predicted_class}")
print(f"Confidence: {confidence}")

Predicted class: Forest
Confidence: [4.2478267e-05 9.9995506e-01 2.4740459e-06]
